In [1]:
from torchvision.models import resnet101

from Models import PatientClassifier, Attention, ResNet101Meta
from PatientDataset import group_split_df, transforms, PatientDS, get_df, PatientLevelSampler
from Train import train, plot_train_hist, set_seed, plot_train_hist
from torch.utils.data import DataLoader
from Evaluate import evaluate, plot_confusion_matrix, probability_histogram
from torch import nn
import torch

In [2]:
# TODO: pip freeze > requirements.txt
# TODO: 2019 data
# TODO: generate confusion matrix for LB1; this model has likely memorized our test set!
# TODO: Check if sampler is working
# Remember to specify the image path in dataset

In [3]:
# Find accelerator if available
acc = torch.accelerator.current_accelerator()
device = acc.type
torch.device(acc) # set device

device(type='mps')

In [4]:
# # Load and Preprocess Data
# set_seed()
# df = get_df("ISIC_2020_Training_GroundTruth_v2.csv")
#
# train_df, val_df, test_df = group_split_df(df, test_size=0.2)
# train_transform, val_transform, test_transform = transforms(img_size=224)
#
# train_ds = PatientDS(train_df, train_transform, image_path = 'training_data/2020/jpg', use_meta=True, upsampling = True, mode='Train', saving_images=False)
# val_ds = PatientDS(val_df, val_transform, image_path = 'training_data/2020/pth', use_meta=False, mode='eval', saving_images=False)
# test_ds = PatientDS(test_df, test_transform, image_path = 'training_data/2020/pth', use_meta=False, mode='eval', saving_images=False)
#
# sampler = PatientLevelSampler(train_ds, pos_fraction=0.3)
# sampler_test = PatientLevelSampler(test_ds, pos_fraction=.3)
#
# train_dl = DataLoader(train_ds, batch_size=1,  sampler=sampler, shuffle = False, num_workers=3)
# val_dl = DataLoader(val_ds, batch_size=1,  sampler=None, shuffle=False, num_workers=0)
# test_dl = DataLoader(test_ds, batch_size=1,  sampler=None, shuffle=False, num_workers=0)

In [5]:
set_seed()
df = get_df("ISIC_2020_Training_GroundTruth_v2.csv")

_, val_df, test_df = group_split_df(df, test_size=0.2)
train_transform, val_transform, _ = transforms(img_size=224)

val_ds = PatientDS(val_df, val_transform, image_path = 'training_data/2020/pth', use_meta=False, mode='eval', saving_images=False)
train_ds = PatientDS(test_df, train_transform, image_path = 'training_data/2020/pth', use_meta=False, mode='train', upsampling = True, saving_images=False)

sampler = PatientLevelSampler(train_ds, pos_fraction=0.3)

val_dl = DataLoader(val_ds, batch_size=1,  sampler=None, shuffle=False, num_workers=0)
train_dl = DataLoader(train_ds, batch_size=1,  sampler=sampler, shuffle=False, num_workers=0)

In [6]:
# NOTE: I am training with the test_dl for now to reduce wait time while I test"
Train_Metrics = {}

for lam in [1,0.1,0.01,0.001,0.0001]:
    print(lam)
    for pos_weight in [1,5,10,20]:
        net = PatientClassifier(Attention, ResNet101Meta, meta_dim = 0)
        # Disable backbone layers
        for p in net.attention.parameters():
            p.requires_grad = True
        for p in net.resnet101.parameters():
            p.requires_grad = False
        for p in net.resnet101.backbone.layer4.parameters():
            p.requires_grad = True
        for p in net.resnet101.head1.parameters():
            p.requires_grad = True
        for p in net.resnet101.head2.parameters():
            p.requires_grad = True
        # Optimizer
        # opt = torch.optim.AdamW( lr=0.0001, params=attention.parameters() )
        opt = torch.optim.AdamW([
            {"params": net.attention.parameters(), "lr": 1e-3},
            {"params": net.resnet101.backbone.layer4.parameters(), "lr": 1e-4},
            {"params": net.resnet101.head1.parameters(), "lr": 1e-4},
            {"params": net.resnet101.head2.parameters(), "lr": 1e-3}
        ])
        # pos_weight=torch.tensor(1.4, device = device, dtype = torch.float32)
        # pos_weight=torch.tensor(22, device = device, dtype = torch.float32)
        criterion_patient = nn.BCEWithLogitsLoss()
        criterion_lesion = nn.BCEWithLogitsLoss(pos_weight=torch.tensor(pos_weight, device = device, dtype = torch.float32))
        print('pos_weight', pos_weight)
        train_metrics = train(
                            net, train_dl, val_dl,
                            criterion_patient, criterion_lesion, opt, lam = lam,
                            epochs=2, device="mps")
        Train_Metrics[f'lam{lam} pos_weight {pos_weight}'] = train_metrics

1
pos_weight 1


100%|██████████| 206/206 [01:11<00:00,  2.89it/s]


Epoch 1: Train Loss=1.0521, Val Loss=0.8072, patient_acc=0.8155, patient_pos_acc=0.0000, lesion_acc=0.9711, lesion_pos_acc=0.0000, 


100%|██████████| 206/206 [01:42<00:00,  2.02it/s]


Epoch 2: Train Loss=0.5399, Val Loss=1.0667, patient_acc=0.8155, patient_pos_acc=0.0000, lesion_acc=0.9712, lesion_pos_acc=0.0000, 
Best validation positive accuracy: 0.0000
pos_weight 5


...

Epoch 1: Train Loss=1.5771, Val Loss=1.1165, patient_acc=0.8155, patient_pos_acc=0.0000, lesion_acc=0.9708, lesion_pos_acc=0.0000, 


100%|██████████| 206/206 [13:37<00:00,  3.97s/it]  


Saved new best model at epoch 2 with patient_auc=0.1991
Epoch 2: Train Loss=1.0281, Val Loss=2.6857, patient_acc=0.2039, patient_pos_acc=0.8158, lesion_acc=0.9708, lesion_pos_acc=0.0000, 
Best validation positive accuracy: 0.8158
pos_weight 10


 21%|██        | 43/206 [01:03<04:00,  1.47s/it]


KeyboardInterrupt: 

In [ ]:
# NOTE: I am training with the test_dl for now to reduce wait time while I test"
Train_Metrics = {}

for lam in [0.1,0.01,0.001,0.0001]:
    print(lam)
    for pos_weight in [1,5,10,20]:
        net = PatientClassifier(Attention, ResNet101Meta, meta_dim = 0)
        # Disable backbone layers
        for p in net.attention.parameters():
            p.requires_grad = True
        for p in net.resnet101.parameters():
            p.requires_grad = False
        for p in net.resnet101.backbone.layer4.parameters():
            p.requires_grad = True
        for p in net.resnet101.head1.parameters():
            p.requires_grad = True
        for p in net.resnet101.head2.parameters():
            p.requires_grad = True
        # Optimizer
        # opt = torch.optim.AdamW( lr=0.0001, params=attention.parameters() )
        opt = torch.optim.AdamW([
            {"params": net.attention.parameters(), "lr": 1e-3},
            {"params": net.resnet101.backbone.layer4.parameters(), "lr": 1e-4},
            {"params": net.resnet101.head1.parameters(), "lr": 1e-4},
            {"params": net.resnet101.head2.parameters(), "lr": 1e-3}
        ])
        # pos_weight=torch.tensor(1.4, device = device, dtype = torch.float32)
        # pos_weight=torch.tensor(22, device = device, dtype = torch.float32)
        criterion_patient = nn.BCEWithLogitsLoss()
        criterion_lesion = nn.BCEWithLogitsLoss(pos_weight=torch.tensor(pos_weight, device = device, dtype = torch.float32))
        print('pos_weight', pos_weight)
        train_metrics = train(
                            net, train_dl, val_dl,
                            criterion_patient, criterion_lesion, opt, lam = lam,
                            epochs=2, device="mps")
        Train_Metrics[f'lam{lam} pos_weight {pos_weight}'] = train_metrics

In [11]:
state_dict = torch.load("best.pt", map_location=device)
net.load_state_dict(state_dict)

outputs = evaluate(net, test_dl, device = device, use_meta= True, threshold=0.5)

NameError: name 'test_dl' is not defined

In [ ]:
outputs.keys()

In [ ]:
plot_confusion_matrix(outputs['preds_patient'], outputs['ys_patient'], output_dir = None, title="Patient_Confusion_Matrix")

In [ ]:
probability_histogram(outputs['ps_patient'], outputs['ys_patient'], output_dir = None, title="Probability_Histogram")

In [ ]:
next(iter(test_dl))

In [ ]:
probability_histogram(outputs['ps_patient'], outputs['ys_patient'], output_dir = None, title="Probability_Histogram")